# Storytelling with Data! in Altair

by Maisa de Oliveira Fraiz

## Introduction

This project aims to replicate selected examples from Cole Nussbaumer's book, "Storytelling with Data - Let's Practice!", using `Python` library `Altair`. The primary objective is to document the reasoning behind the modifications proposed by the author, while also highlighting the challenges that arise when transitioning from the book's Excel-based approach to programming in a different software environment.

`Altair` was selected for this project due to its declarative syntax, interactivity, grammar of graphics, and compatibility with `Streamlit` and other web formatting tools, while within the user-friendly Python environment. Anticipated challenges include the comparatively smaller documentation and development community of `Altair` compared to more established libraries like `Matplotlib`, `Seaborn`, or `Plotly`, and the difficulty to effectively translate tasks that might appear straightforward in Excel.

In addition to replicating the graphs from the book, the objective is to extend the functionality by creating interactive versions, fully leveraging Altair's capabilities.

## Imports

In [89]:
import pandas as pd
import numpy as np
import altair as alt

## Chapter 2 - Choose an effective visual

*"When I have some data I need to show, how do I do that in an effective way?"* - Cole Nussbaumer

### Exercise 2.1 - improve this table

The data for this exercise can be found in the book's official website: https://www.storytellingwithdata.com/letspractice/downloads

The first problem with the Excel-to-Altair translation arises from the data itself, as it is polluted with titles and texts for readability in Excel. This, however, is not friendly when dealing with Python, so we should be careful when loading it. Alterations like this will happen in all subsequent exercises.



In [90]:
# Example of wrong loading
table = pd.read_excel(r"..\..\Data\2.1 EXERCISE.xlsx")
table

,EXERCISE 2.1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,FIG 2.1a,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,New client tier share,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
6,NaN,A,77,0.070772,4.675,0.25
7,NaN,A+,19,0.017463,3.927,0.21
8,NaN,B,338,0.310662,5.984,0.32
9,NaN,C,425,0.390625,2.805,0.15


In [91]:
del table

In [92]:
# Right loading
table = pd.read_excel(r"..\..\Data\2.1 EXERCISE.xlsx", usecols = [1, 2, 3, 4, 5], header = 6)
table

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
0,A,77,0.070772,4.675,0.25
1,A+,19,0.017463,3.927,0.21
2,B,338,0.310662,5.984,0.32
3,C,425,0.390625,2.805,0.15
4,D,24,0.022059,0.374,0.02


The initial changes recommended in the book focus on improving the table's readability. These changes include reordering the tiers, adding a row to show the total value, incorporating a category called "All others" to account for unmentioned values when the total percentage doesn't add up to 100%, and rounding the numbers while adjusting the percentage format as required.

The following code implements these modifications.


In [93]:
# Ordering the tiers

table = table.loc[[1, 0, 2, 3, 4]]

In [94]:
# Fixing the percentages

table['% Accounts'] = table['% Accounts'].apply(lambda x: x*100)
table['% Revenue'] = table['% Revenue'].apply(lambda x: x*100)

In [95]:
# Calculating and adding "All other" values

other_account_per = 100 - table['% Accounts'].sum()
other_revenue_per = 100 - table['% Revenue'].sum()

other_account_num = (other_account_per*table['# of Accounts'][0])/table['% Accounts'][0]
other_revenue_num = (other_revenue_per*table['Revenue ($M)'][0])/table['% Revenue'][0]

table.loc[len(table)] = ["All other", other_account_num, other_account_per, other_revenue_num, other_revenue_per]


In [96]:
# Since we will use not-rounded values or the total row for the graphs,
# we should create a new variable before making the following alterations

table_charts = table.copy()

In [97]:
# Adding total values row

table.loc[len(table)] = ["Total", table['# of Accounts'].sum(), table['% Accounts'].sum(),
                        table['Revenue ($M)'].sum(), table['% Revenue'].sum()]

In [98]:
# Rounding the numbers

table['% Accounts'] = table['% Accounts'].apply(lambda x: round(x))
table['Revenue ($M)'] = table['Revenue ($M)'].apply(lambda x: round(x, 1))

The new table is as follows:

In [100]:
table

,Tier,# of Accounts,% Accounts,Revenue ($M),% Revenue
1,A+,19.0,2,3.9,21.0
0,A,77.0,7,4.7,25.0
2,B,338.0,31,6.0,32.0
3,C,425.0,39,2.8,15.0
4,D,24.0,2,0.4,2.0
5,All other,205.0,19,0.9,5.0
6,Total,1088.0,100,18.7,100.0


or, for even better readability in `Python`:

In [101]:
table.set_index("Tier")

,# of Accounts,% Accounts,Revenue ($M),% Revenue
Tier,,,,
A+,19.0,2,3.9,21.0
A,77.0,7,4.7,25.0
B,338.0,31,6.0,32.0
C,425.0,39,2.8,15.0
D,24.0,2,0.4,2.0
All other,205.0,19,0.9,5.0
Total,1088.0,100,18.7,100.0


Some changes were not implemented, such as colors of rows, alignment of text, and embedding graphs into the table, for lack of compatibility with the Pandas DataFrame format. The percentage symbol (%) next to the number in the percentage columns wasn't added since doing this in Python will transform the data from `int` to `string`, and therefore is not a recommended approach.

Considering that percentages depict a fraction of a whole, the next proposal is to employ a pie chart. 
Here is the default Altair graph version:

In [102]:
# Default pie chart

alt.Chart(table_charts).mark_arc().encode(
    theta = "% Accounts",
    color = alt.Color('Tier'),
)

alt.Chart(...)

Some of the adjustments needed to bring it closer to the original include reordering the tiers, changing the labels position, altering the color palette, and adding an title.

In [114]:
## % of Accounts Pie Chart

base = alt.Chart(table_charts, title = alt.Title(r"% of Total Accounts", anchor = 'start', fontWeight = 'normal')).encode(
    theta = alt.Theta("% Accounts:Q").stack(True),
    color = alt.Color('Tier').legend(None),
    order = alt.Order(field = 'Tier'))



pie = base.mark_arc(outerRadius = 115)
text = base.mark_text(radius = 140, size = 15).encode(text = alt.Text("Tier"))


acc_pie = pie + text
acc_pie

alt.LayerChart(...)

Not informing the data type for the field `order` makes it so Altair rearranges the `Tiers` alphabetically instead of using the order provided by dataframe. We can fix this by identifying `Tier` as Ordered (O).

In [123]:
## % of Accounts Pie Chart

# 'anchor = start' aligns the title to the left
# 'fontWeight = normal' is used to counter the default bold title setting in Altair

base = alt.Chart(table_charts, title = alt.Title(r"% of Total Accounts", anchor = 'start', fontWeight = 'normal')).encode(
    theta = alt.Theta("% Accounts:Q").stack(True),
    color = alt.Color('Tier',
                      scale = alt.Scale(range = ['#4d71bc', '#5d9bd4', '#6fae45', '#febf0f', '#e77e2d', '#a6a6a6']),
                      sort = None #so that the colors don't follow the alphabetic order
                      ).legend(None),
    order = alt.Order(field = 'Tier:O'))



pie = base.mark_arc(outerRadius = 115)
text = base.mark_text(radius = 140, size = 15).encode(text = alt.Text("Tier"))


acc_pie = pie + text
acc_pie

alt.LayerChart(...)

Initially, `offset` was used instead of `anchor`, defining the position of the title in the x-axis manually by pixels. However, anchoring it to the start is a quicker and more clean approach.

The HEX color code values of the palette from the book were acquired through the use of the online tool "Color Picker Online," which is freely accessible at https://imagecolorpicker.com/.

The pie chart above can now be easily modified to represent the percentage of total revenue.

In [228]:
# % of Revenue Pie Chart

base = alt.Chart(table_charts, title = alt.Title(r"% of Total Revenue",  anchor = 'start', fontWeight = 'normal')).encode(
    theta = alt.Theta("% Revenue:Q").stack(True),
    color = alt.Color('Tier',
                      scale = alt.Scale(range = ['#4d71bc', '#5d9bd4', '#6fae45', '#febf0f', '#e77e2d', '#a6a6a6']),
                      sort = None
                      ).legend(None),
    order = alt.Order(field ='Tier:O'))



pie = base.mark_arc(outerRadius = 115)
text = base.mark_text(radius = 135, size = 14, align = "left").encode(text = alt.Text("Tier"))


rev_pie = pie + text
rev_pie 

alt.LayerChart(...)

With both graphs available, we can add them next to each other and include a main title.

In [225]:
# Finished Pie Chart

pies = acc_pie | rev_pie

pies.properties(
    title = alt.Title('New Client Tier Share', offset = 10, fontSize = 20)
)



alt.HConcatChart(...)

![Alt text](\Images\2_1e.png)

Pie charts can present readability challenges, as the human eye struggles to differentiate the relative volumes of slices effectively. While adding data percentages next to the slices can enhance comprehension, it may also introduce unnecessary clutter to the visualization.

The next graph proposed to tackle is a horizontal bar chart. Since now the comparison does not involve angles and are aligned at the start point, discerning the segment's scale is easier.

This is the default representation in Altair:

In [129]:
# Default altair bar chart

alt.Chart(table_charts).mark_bar().encode(
    y = alt.Y('Tier'),
    x = alt.X('% Accounts'))

alt.Chart(...)

The necessary adjustments involve placing the "Tier" label in the upper left corner, displaying values next to the bars instead of using an x-axis, and adding a title while rearranging the tiers.

In [140]:
# Alterations as per book

base = alt.Chart(table_charts, title = alt.Title('TIER | % OF TOTAL ACCOUNTS', anchor = 'start', fontWeight = 'normal')
).mark_bar().encode(
    y = alt.Y('Tier',  title = None),
    x = alt.X('% Accounts').axis(None),
    order = alt.Order(field = 'Tier:O'),
    text = alt.Text("% Accounts", format = ".0f"))

final_acc = base.mark_bar() + base.mark_text(align = 'left', dx = 2)
final_acc

alt.LayerChart(...)

Adding the `order` by `Tier:O` didn't had the same effect as it did on the pie chart. The compatible method for this case is adding a `sort` keyword in the axis to be sorted.

In [171]:
base = alt.Chart(table_charts, title = alt.Title('TIER   | % OF TOTAL ACCOUNTS     |', anchor = 'start', fontWeight = 'normal')).encode(
    y = alt.Y('Tier', sort = ["A+"], title = None),
    x = alt.X('% Accounts').axis(None),
    text = alt.Text("% Accounts", format = ".0f"))

final_acc = (base.mark_bar() + base.mark_text(align = 'left', dx = 2)).properties(width = 150)
final_acc

alt.LayerChart(...)

Now we do the same for the revenue column.

* In addition, the y-axis is removed. This is so the axis isn't repeated when uniting the charts.

In [162]:
base = alt.Chart(table_charts, title = alt.Title('% OF TOTAL REVENUE', anchor = 'start', fontWeight = 'normal')).mark_bar().encode(
    y = alt.Y('Tier', sort = ["A+"]).axis(None),
    x = alt.X('% Revenue').axis(None),
    text = alt.Text("% Revenue", format = ".0f"))

final_rev = (base.mark_bar() + base.mark_text(align = 'left', dx = 2)).properties(width = 150)
final_rev

alt.LayerChart(...)

Similar to the pie chart, we can arrange these graphs side by side and include a main title.

In [172]:
final = final_acc | final_rev
final.configure_view(stroke=None).properties(
    title = alt.Title('New Client Tier Share', anchor = 'start', fontSize = 20)
)

alt.HConcatChart(...)

![Alt text](\Images\2_1f.png)

It both the pie and bar chart, the labeling beside the value is not in the same position as the examples provided. This discrepancy arises from the fact that adjusting these labels to match the book's examples, with variations in positions (some inside and some outside of the pie), different colors, and even omitting some labels, would be a labor-intensive manual task in Altair. These adjustments are primarily for aesthetic purposes and do not significantly impact readability, in some cases even obscuring the information being presented. 

Examples of how to manually define labels will be presented in future exercises.

The two graphs can be merged into a single grouped bar chart.

In [175]:
# Default by Altair

alt.Chart(table_charts).mark_bar().encode(
    x = alt.X('value:Q'),
    y = alt.Y('variable:N'),
    color = alt.Color('variable:N', legend = alt.Legend(title = 'Metric')),
    row = alt.Row(
                'Tier:O'
                )
).transform_fold(
    fold = ['% Accounts', '% Revenue'],
    as_ = ['variable', 'value']
)

alt.Chart(...)

The necessary alterations involve removing the grid, adjusting label positions and reducing redundancy, adding a title and subtitle, and changing the color palette.

In [283]:
# Proper alterations

alt.Chart(table_charts, title = alt.Title('New client tier share', fontSize = 20)).mark_bar().encode(
    x = alt.X('value:Q', axis = alt.Axis(title = "TIER |  % OF TOTAL ACCOUNTS vs REVENUE", 
                                         grid = False, 
                                         orient = 'top', 
                                         labelColor = "#888888", 
                                         titleColor = '#888888')),
    y = alt.Y('variable:N', axis = alt.Axis(title = None, labels = False, ticks = False)),
    color = alt.Color('variable:N', 
                      legend = alt.Legend(title = 'Metric'),
                      scale = alt.Scale(range = ['#b4c6e4', '#4871b7'])
                      ),
    row = alt.Row(
                'Tier:O', 
                header = alt.Header(labelAngle = 0, labelAlign = "left"), 
                title = None,
                sort = ['A+'],
                spacing = 10
                )
).transform_fold(
    fold = ['% Accounts', '% Revenue'],
    as_ = ['variable', 'value']
).properties(
    width = 200
).configure_view(stroke = None) 

alt.Chart(...)

![Alt text](\Images\2_1g.png)

We should now modify this chart to be in a vertical orientation. This can be done by switching the y and x axis and the "Row" class to the "Column" class, as well as reorient the labels.

In [282]:
alt.Chart(table_charts, title = alt.Title('New client tier share', fontSize = 20)).mark_bar().encode(
    y = alt.Y('value:Q', axis = alt.Axis(title = "% OF TOTAL ACCOUNTS vs REVENUE",
                                         titleAlign ='left',
                                         titleAngle = 0,
                                         titleAnchor = 'end',
                                         titleY = -10,
                                         grid = False, 
                                         labelColor = "#888888", 
                                         titleColor = '#888888')),
    x = alt.X('variable:N', axis = alt.Axis(title = None, labels = False, ticks = False)),
    color = alt.Color('variable:N', 
                      legend = alt.Legend(title = 'Metric'),
                      scale = alt.Scale(range = ['#b4c6e4', '#4871b7'])
                      ),
    column = alt.Column(
        'Tier:O', 
        header = alt.Header(labelOrient = 'bottom', titleOrient = "bottom", titleAnchor = "start"),
        sort = ['A+'],
        title = 'TIER'
        )
).transform_fold(
    fold = ['% Accounts', '% Revenue'],
    as_ = ['variable', 'value']
).properties(
    width = 50
).configure_view(stroke = None)


alt.Chart(...)

![Alt text](\Images\2_1h.png)

It's worth noting that titles in Altair do not readily support the option of changing the colors of individual words within them. As a simple solution for the time being, we will retain the legend that effectively indicates which column corresponds to each word. In a notebook, you can work around this by using ``Latex`` in a ``Markdown`` cell, but it won't be seamlessly integrated with the chart. It would look something like this:

**New client tier share**


"% OF TOTAL $\textcolor{#b4c6e4}{ACCOUNTS}$ vs $\textcolor{#4871b7}{REVENUE}$"

Future exercises will delve into a more complicated way to tackle this challenge.


In the code above, we've utilized the `transform_fold` method to generate the grouped bar chart because our data is structured in the 'wide form', which is the standard Excel format. However, Altair (as well as other visualization languages) is inherently designed to work with 'long form' data. The `transform_fold` function automates this conversion within the chart, enabling us to create the graph. This approach can obscure the process, making it preferable to perform the data transformation before creating the visualizations.

In [274]:
# Transforms the data to the long-form format.

melted_table = pd.melt(table_charts, id_vars = ['Tier'], var_name = 'Metric', value_name = 'Value')
melted_table

,Tier,Metric,Value
0,A+,# of Accounts,19.000000
1,A,# of Accounts,77.000000
2,B,# of Accounts,338.000000
3,C,# of Accounts,425.000000
4,D,# of Accounts,24.000000
5,All other,# of Accounts,205.000000
6,A+,% Accounts,1.746324
7,A,% Accounts,7.077206
8,B,% Accounts,31.066176
9,C,% Accounts,39.062500


We can now use this table to remake the bar chart without the ``transform_fold`` method.

In [285]:
selected_rows = melted_table[melted_table['Metric'].isin(['% Accounts', '% Revenue'])]

alt.Chart(selected_rows, title = alt.Title('New client tier share', fontSize = 20)).mark_bar().encode(
    y = alt.Y('Value', axis = alt.Axis(title = "% OF TOTAL ACCOUNTS vs REVENUE",
                                         titleAlign ='left',
                                         titleAngle = 0,
                                         titleAnchor = 'end',
                                         titleY = -10,
                                         grid = False, 
                                         labelColor = "#888888", 
                                         titleColor = '#888888')),
    x = alt.X('Metric', axis = alt.Axis(title = None, labels = False, ticks = False)),
    color = alt.Color('Metric', scale = alt.Scale(range = ['#b4c6e4', '#4871b7'])),
    column = alt.Column('Tier',
                        header = alt.Header(labelOrient = 'bottom', titleOrient = "bottom", titleAnchor = "start"),
                        sort = ['A+'],
                        title = 'TIER'
                        )
    ).properties(
        height = 200, width = 50
).configure_view(stroke = None)


alt.Chart(...)

The next proposed graph is an extension of the previous bar chart, featuring the addition of lines to accentuate the endpoints of the columns within the same tier.

However, due to the nature of faceted charts, we encounter an error (*ValueError: Faceted charts cannot be layered. Instead, layer the charts before faceting*) when attempting to layer it. This issue arises because, in faceted charts, the x-axis structure is altered. 

Now that we've transformed our data into long-format, we can work around this problem by creating our graph without using the 'column' method, and thereby, avoiding faceting. Instead of specifying 'x' as 'Metric,' 'y' as 'Value,' 'color' as 'Metric,' and 'column' as 'Tier,' we can redefine 'x' as 'Tier,' 'y' as 'Value,' 'color' as 'Metric,' and introduce 'XOffset' for controlling the horizontal positioning of data points within a group. In essence, 'column' primarily serves to define distinct x-axis categories, while 'XOffset' is employed to manage the horizontal placement of data points within a group.

The following chart incorporates the alterations we discussed and yields a graph that closely resembles the previous one.

In [306]:
# New bar chart

bar = alt.Chart(selected_rows, title = alt.Title('New client tier share', fontSize = 20, anchor = 'start')).mark_bar().encode(
    x = alt.X('Tier', axis = alt.Axis(title = 'TIER',
                                      labelAngle = 0, 
                                      titleAnchor = "start", 
                                      domain = False,
                                      ticks = False), sort = ['A+']),
    y = alt.Y('Value', axis = alt.Axis(title = "% OF TOTAL ACCOUNTS vs REVENUE",
                                       titleAlign ='left',
                                       titleAngle = 0,
                                       titleAnchor = 'end',
                                       titleY = -10,
                                       grid = False, 
                                       labelColor = "#888888", 
                                       titleColor = '#888888')),
    color = alt.Color('Metric', scale = alt.Scale(range = ['#b4c6e4', '#4871b7'])),
    xOffset = 'Metric'
    ).properties(
        height = 250, width = 375
)

bar.configure_view(stroke = None)

alt.Chart(...)

Now, we can layer the graph and introduce the lines. It's worth noting that creating the lines in Altair is not a straightforward task and a considerable amount of documentation searching was necessary to achieve it.

In [307]:
# x, y and y2 do not accept to be defined as "condition", so repetitive code is necessary

# Lines that are ascending
rule_asc = alt.Chart(selected_rows).mark_rule(x2Offset = 10, xOffset = -10
).encode(
    x = alt.X('Tier', sort = ['A+']),
    x2 = alt.X2('Tier'),
    y = alt.Y('min(Value)'),
    y2 = alt.Y2('max(Value)'),
    strokeWidth = alt.value(2), 
    opacity = alt.condition(
        (alt.datum.Tier == 'A+') | 
        (alt.datum.Tier == 'A')  |
        (alt.datum.Tier == 'B'), 
        alt.value(1), alt.value(0)
        )
    )

# Lines that are descending
rule_desc = alt.Chart(selected_rows).mark_rule(x2Offset = 10, xOffset = -10
).encode(
    x = alt.X('Tier', sort = ['A+']),
    x2 = alt.X2('Tier'),
    y = alt.Y('max(Value)'),
    y2 = alt.Y2('min(Value)'),
    strokeWidth = alt.value(2), 
    opacity = alt.condition(
        (alt.datum.Tier == 'A+') | 
        (alt.datum.Tier == 'A')  |
        (alt.datum.Tier == 'B'), 
        alt.value(0), alt.value(1)
        )
    )

# Points of % Revenue where % Revenue > % Accounts
points1 = alt.Chart(selected_rows).mark_point(filled = True, xOffset = 10, color = "black").encode(
    x = alt.X('Tier', sort = ['A+']),
    y = alt.Y('max(Value)'),
    opacity = alt.condition(
        (alt.datum.Tier == 'A+') | 
        (alt.datum.Tier == 'A')  |
        (alt.datum.Tier == 'B'), 
        alt.value(1), alt.value(0)
        )
    )

# Points of % Revenue where % Revenue < % Accounts
points2 = alt.Chart(selected_rows).mark_point(filled = True, xOffset = 10, color = "black").encode(
    x = alt.X('Tier', sort = ['A+']),
    y = alt.Y('min(Value)'),
    opacity = alt.condition(
        (alt.datum.Tier == 'A+') | 
        (alt.datum.Tier == 'A')  |
        (alt.datum.Tier == 'B'), 
        alt.value(0), alt.value(1)
        )
    )

# Points of % Accounts where % Revenue < % Accounts
points3 = alt.Chart(selected_rows).mark_point(filled = True, xOffset = -10, color = "black").encode(
    x = alt.X('Tier', sort = ['A+']),
    y = alt.Y('max(Value)'),
    opacity = alt.condition(
        (alt.datum.Tier == 'A+') | 
        (alt.datum.Tier == 'A')  |
        (alt.datum.Tier == 'B'), 
        alt.value(0), alt.value(1)
        )
    )

# Points of % Revenue where % Revenue > % Accounts
points4 = alt.Chart(selected_rows).mark_point(filled = True, xOffset = -10, color = "black").encode(
    x = alt.X('Tier', sort = ['A+']),
    y = alt.Y('min(Value)'),
    opacity = alt.condition(
        (alt.datum.Tier == 'A+') | 
        (alt.datum.Tier == 'A')  |
        (alt.datum.Tier == 'B'), 
        alt.value(1), alt.value(0)
        )
    )

final = bar + rule_asc + rule_desc + points1 + points2 + points3 + points4
final.configure_view(stroke = None)


alt.LayerChart(...)

![Alt text](\Images\2_1i.png)

In [322]:
final.configure_mark(opacity = 0).configure_view(stroke = None).configure_legend(disable = True)

alt.LayerChart(...)

![Alt text](\Images\2_1j.png)

In [356]:

base = alt.Chart(selected_rows, title = alt.Title("New client tier share", anchor = 'start', fontWeight = 'normal', fontSize = 20))

line = base.mark_line(point = True).encode(
    x = alt.X('Metric', axis = alt.Axis(title = None, labelAngle = 0)),
    y = alt.Y('Value', axis = alt.Axis(grid = False, title = "%")),
    color = alt.Color('Tier', 
                      scale = alt.Scale(range = ['black', 'black', 'black', 'black', 'black', 'black']),
                      legend = None)
).properties(
    width = 300,
    height = 350
)


labels = base.mark_text(
    align='left', 
    dx =alt.condition(alt.datum.Metric == '% Accounts', alt.value(10), alt.value(-10)) 
).encode(
    x = alt.X('Metric'),
    y = alt.Y('Value'),
    text = alt.Text('Value:Q', format='.0f'),
)

final = line + labels

final.configure_view(stroke = None)

SchemaValidationError: '{'condition': {'test': "(datum.Metric === '% Accounts')", 'value': 10}, 'value': -10}' is an invalid value for `dx`. Valid values are of type 'number'.

Additional properties are not allowed ('condition', 'value' were unexpected)'expr' is a required property

In [361]:

labels1 = base.mark_text(
    align='left', 
    dx = 10
).encode(
    x = alt.X('Metric'),
    y = alt.Y('Value'),
    text = alt.Text('Value:Q', format='.0f'),
    opacity = alt.condition(alt.datum.Metric == '% Accounts', alt.value(0), alt.value(1)) 
)

labels2 = base.mark_text(
    align='left', 
    dx = -20
).encode(
    x = alt.X('Metric'),
    y = alt.Y('Value'),
    text = alt.Text('Value:Q', format='.0f'),
    opacity = alt.condition(alt.datum.Metric == '% Accounts', alt.value(1), alt.value(0)) 
)

final = line + labels1 + labels2

final.configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\2_1k.png)